In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score, accuracy_score

# загрузка и просмотр данных

In [2]:
train = pd.read_csv('train.csv')
train

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date,successful_utilization,treatment
0,NaN,1.0,Representatives,123.0,Первичная ДК,1,14.0,147000.0,PLT,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-07-04,0,1
1,NaN,1.0,Offline,43.0,Airports,0,1.0,120000.0,PLT,0.0,...,1.0,0,1.0,0.0,-1.2,1.0,0,2024-06-06,0,1
2,0.0,1.0,Web,2.0,seo,0,NaN,15000.0,PLT,0.0,...,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-21,1,1
3,0.0,1.0,MB,2.0,One Click Offer,0,91.0,260000.0,PLT,0.0,...,1.0,0,NaN,1.0,-1.2,1.0,0,2024-05-23,0,1
4,0.0,1.0,Representatives,123.0,Первичная ДК,1,1.0,130000.0,PLT,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-28,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297245,NaN,1.0,Representatives,123.0,Первичная ДК,1,34.0,95000.0,PLT,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-13,1,1
297246,NaN,1.0,Representatives,123.0,Первичная ДК,1,1.0,83000.0,PLT,0.0,...,NaN,0,1.0,0.0,-1.2,NaN,0,2024-07-08,0,1
297247,0.0,1.0,Mobile Warm,2.0,TOOS cor clients,0,2.0,170000.0,PLT,0.0,...,1.0,0,NaN,0.0,1.0,1.0,0,2024-06-12,0,1
297248,0.0,1.0,Representatives,123.0,Первичная ДК,1,5.0,15000.0,PLT,0.0,...,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-02,1,1


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297250 entries, 0 to 297249
Columns: 152 entries, application_1 to treatment
dtypes: float64(131), int64(17), object(4)
memory usage: 344.7+ MB


In [4]:
train['retro_date'] = pd.to_datetime(train['retro_date'])
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297250 entries, 0 to 297249
Columns: 152 entries, application_1 to treatment
dtypes: datetime64[ns](1), float64(131), int64(17), object(3)
memory usage: 344.7+ MB


In [5]:
train.isna().sum().sum()

9615080

In [6]:
test = pd.read_csv('test.csv')
test

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,bki_46,feature_24,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date
0,0.0,1.0,NaN,NaN,NaN,0,6.0,NaN,NaN,0.0,...,373942.00,0.0,1.0,0,0.0,0.0,-1.2,1.0,0,2024-06-18
1,0.0,1.0,NaN,NaN,NaN,0,45.0,NaN,NaN,0.0,...,0.00,0.0,0.0,0,0.0,1.0,-1.2,1.0,0,2024-05-09
2,0.0,1.0,MB,2.0,Heavy xsell,0,13.0,299000.0,PLT,0.0,...,9724333.00,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-05-30
3,0.0,1.0,Representatives,123.0,SIM,1,158.0,140000.0,PLT,0.0,...,0.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-27
4,0.0,1.0,Representatives,123.0,Первичная ДК,0,36.0,170000.0,PLT,0.0,...,2237284.00,0.0,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74308,NaN,1.0,Representatives,123.0,Multitask,1,NaN,15000.0,PLT,0.0,...,2834099.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-15
74309,0.0,1.0,Representatives,123.0,Первичная ДК,1,12.0,95000.0,PLT,0.0,...,145886.00,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-10
74310,0.0,NaN,PreApprove (Mobile Applicants),2.0,Other,0,5.0,15000.0,PLT,0.0,...,5896358.00,0.0,0.0,1,0.0,1.0,-1.2,1.0,1,2024-06-27
74311,0.0,1.0,Representatives,123.0,Перевыпуск ДК,1,52.0,70000.0,PLT,0.0,...,5339.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-10


In [7]:
test['retro_date'] = pd.to_datetime(test['retro_date'])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74313 entries, 0 to 74312
Columns: 150 entries, application_1 to retro_date
dtypes: datetime64[ns](1), float64(131), int64(15), object(3)
memory usage: 85.0+ MB


In [8]:
sub = pd.read_csv('sample_submission.csv', index_col=0)
sub

,successful_utilization
0,-0.635423
1,-0.884969
2,0.000742
3,0.030837
4,-0.355528
...,...
74308,0.640732
74309,0.933332
74310,-0.085592
74311,-0.886694


# создание модели и подготовка данных для нее

## подготовка данных

In [9]:
Counter(train.treatment)

Counter({1: 282482, 0: 14768})

In [10]:
cat_train = ['cc_2', 'cc_4', 'cc_6']

In [11]:
lst=[]
for i in cat_train:
    for j in train[i]:
        lst.append(j)
lst

['Representatives',
 'Offline',
 'Web',
 'MB',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Down-sell',
 'Representatives',
 'Representatives',
 'Representatives',
 'MB',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Mobile Warm',
 'Representatives',
 'Representatives',
 'Representatives',
 'MB',
 'Web',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'MB',
 'Web',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'MB',
 'Representatives',
 'Mobile Warm',
 'Representatives',
 'PreApprove (Mobile Applicants)',
 'Representatives',
 'Representatives',
 'MB',
 nan,
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'Representatives',
 'PreApprove (M

In [12]:
le = LabelEncoder()

le.fit(lst)

for col in cat_train:
    train[col] = le.transform(train[col])

train

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date,successful_utilization,treatment
0,NaN,1.0,45,123.0,89,1,14.0,147000.0,36,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-07-04,0,1
1,NaN,1.0,32,43.0,5,0,1.0,120000.0,36,0.0,...,1.0,0,1.0,0.0,-1.2,1.0,0,2024-06-06,0,1
2,0.0,1.0,58,2.0,75,0,NaN,15000.0,36,0.0,...,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-21,1,1
3,0.0,1.0,26,2.0,33,0,91.0,260000.0,36,0.0,...,1.0,0,NaN,1.0,-1.2,1.0,0,2024-05-23,0,1
4,0.0,1.0,45,123.0,89,1,1.0,130000.0,36,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-28,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297245,NaN,1.0,45,123.0,89,1,34.0,95000.0,36,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-13,1,1
297246,NaN,1.0,45,123.0,89,1,1.0,83000.0,36,0.0,...,NaN,0,1.0,0.0,-1.2,NaN,0,2024-07-08,0,1
297247,0.0,1.0,29,2.0,55,0,2.0,170000.0,36,0.0,...,1.0,0,NaN,0.0,1.0,1.0,0,2024-06-12,0,1
297248,0.0,1.0,45,123.0,89,1,5.0,15000.0,36,0.0,...,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-02,1,1


In [13]:
X = train.drop(['successful_utilization'], axis = 1)
X

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,feature_24,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date,treatment
0,NaN,1.0,45,123.0,89,1,14.0,147000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-07-04,1
1,NaN,1.0,32,43.0,5,0,1.0,120000.0,36,0.0,...,0.0,1.0,0,1.0,0.0,-1.2,1.0,0,2024-06-06,1
2,0.0,1.0,58,2.0,75,0,NaN,15000.0,36,0.0,...,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-21,1
3,0.0,1.0,26,2.0,33,0,91.0,260000.0,36,0.0,...,NaN,1.0,0,NaN,1.0,-1.2,1.0,0,2024-05-23,1
4,0.0,1.0,45,123.0,89,1,1.0,130000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-28,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297245,NaN,1.0,45,123.0,89,1,34.0,95000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-13,1
297246,NaN,1.0,45,123.0,89,1,1.0,83000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,NaN,0,2024-07-08,1
297247,0.0,1.0,29,2.0,55,0,2.0,170000.0,36,0.0,...,NaN,1.0,0,NaN,0.0,1.0,1.0,0,2024-06-12,1
297248,0.0,1.0,45,123.0,89,1,5.0,15000.0,36,0.0,...,0.0,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-02,1


In [14]:
y = train.successful_utilization.values
y

array([0, 0, 1, ..., 0, 1, 1], dtype=int64)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y)

## random forest

## catboost

In [16]:
# cat = CatBoostClassifier(iterations=20000, od_wait=1000, task_type='GPU')
cat = CatBoostClassifier(iterations=1000, task_type='GPU')
cat.fit(X_train, y_train, verbose=200, eval_set=(X_test, y_test))

Learning rate set to 0.044725
0:	learn: 0.6716034	test: 0.6716482	best: 0.6716482 (0)	total: 19.7ms	remaining: 19.7s
200:	learn: 0.4818145	test: 0.4851571	best: 0.4851571 (200)	total: 2.33s	remaining: 9.26s
400:	learn: 0.4733212	test: 0.4806932	best: 0.4806932 (400)	total: 4.59s	remaining: 6.86s
600:	learn: 0.4673876	test: 0.4789603	best: 0.4789603 (600)	total: 6.89s	remaining: 4.57s
800:	learn: 0.4621620	test: 0.4779927	best: 0.4779913 (798)	total: 9.37s	remaining: 2.33s
999:	learn: 0.4575164	test: 0.4773378	best: 0.4773349 (998)	total: 11.6s	remaining: 0us
bestTest = 0.4773348796
bestIteration = 998
Shrink model to first 999 iterations.


# создание предикта

In [17]:
model = cat

In [18]:
test 

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,bki_46,feature_24,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date
0,0.0,1.0,NaN,NaN,NaN,0,6.0,NaN,NaN,0.0,...,373942.00,0.0,1.0,0,0.0,0.0,-1.2,1.0,0,2024-06-18
1,0.0,1.0,NaN,NaN,NaN,0,45.0,NaN,NaN,0.0,...,0.00,0.0,0.0,0,0.0,1.0,-1.2,1.0,0,2024-05-09
2,0.0,1.0,MB,2.0,Heavy xsell,0,13.0,299000.0,PLT,0.0,...,9724333.00,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-05-30
3,0.0,1.0,Representatives,123.0,SIM,1,158.0,140000.0,PLT,0.0,...,0.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-27
4,0.0,1.0,Representatives,123.0,Первичная ДК,0,36.0,170000.0,PLT,0.0,...,2237284.00,0.0,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74308,NaN,1.0,Representatives,123.0,Multitask,1,NaN,15000.0,PLT,0.0,...,2834099.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-15
74309,0.0,1.0,Representatives,123.0,Первичная ДК,1,12.0,95000.0,PLT,0.0,...,145886.00,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-10
74310,0.0,NaN,PreApprove (Mobile Applicants),2.0,Other,0,5.0,15000.0,PLT,0.0,...,5896358.00,0.0,0.0,1,0.0,1.0,-1.2,1.0,1,2024-06-27
74311,0.0,1.0,Representatives,123.0,Перевыпуск ДК,1,52.0,70000.0,PLT,0.0,...,5339.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-10


In [19]:
for col in cat_train:
    test[col] = le.transform(test[col])
test

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,bki_46,feature_24,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date
0,0.0,1.0,68,NaN,68,0,6.0,NaN,68,0.0,...,373942.00,0.0,1.0,0,0.0,0.0,-1.2,1.0,0,2024-06-18
1,0.0,1.0,68,NaN,68,0,45.0,NaN,68,0.0,...,0.00,0.0,0.0,0,0.0,1.0,-1.2,1.0,0,2024-05-09
2,0.0,1.0,26,2.0,16,0,13.0,299000.0,36,0.0,...,9724333.00,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-05-30
3,0.0,1.0,45,123.0,49,1,158.0,140000.0,36,0.0,...,0.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-27
4,0.0,1.0,45,123.0,89,0,36.0,170000.0,36,0.0,...,2237284.00,0.0,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74308,NaN,1.0,45,123.0,30,1,NaN,15000.0,36,0.0,...,2834099.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-15
74309,0.0,1.0,45,123.0,89,1,12.0,95000.0,36,0.0,...,145886.00,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-10
74310,0.0,NaN,43,2.0,34,0,5.0,15000.0,36,0.0,...,5896358.00,0.0,0.0,1,0.0,1.0,-1.2,1.0,1,2024-06-27
74311,0.0,1.0,45,123.0,90,1,52.0,70000.0,36,0.0,...,5339.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-10


In [20]:
test_0 = test.copy()
test_0['treatment'] = 0
test_0

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,feature_24,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date,treatment
0,0.0,1.0,68,NaN,68,0,6.0,NaN,68,0.0,...,0.0,1.0,0,0.0,0.0,-1.2,1.0,0,2024-06-18,0
1,0.0,1.0,68,NaN,68,0,45.0,NaN,68,0.0,...,0.0,0.0,0,0.0,1.0,-1.2,1.0,0,2024-05-09,0
2,0.0,1.0,26,2.0,16,0,13.0,299000.0,36,0.0,...,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-05-30,0
3,0.0,1.0,45,123.0,49,1,158.0,140000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-27,0
4,0.0,1.0,45,123.0,89,0,36.0,170000.0,36,0.0,...,0.0,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-08,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74308,NaN,1.0,45,123.0,30,1,NaN,15000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-15,0
74309,0.0,1.0,45,123.0,89,1,12.0,95000.0,36,0.0,...,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-10,0
74310,0.0,NaN,43,2.0,34,0,5.0,15000.0,36,0.0,...,0.0,0.0,1,0.0,1.0,-1.2,1.0,1,2024-06-27,0
74311,0.0,1.0,45,123.0,90,1,52.0,70000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-10,0


In [21]:
test_1 = test.copy()
test_1['treatment'] = 1
test_1

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,feature_24,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date,treatment
0,0.0,1.0,68,NaN,68,0,6.0,NaN,68,0.0,...,0.0,1.0,0,0.0,0.0,-1.2,1.0,0,2024-06-18,1
1,0.0,1.0,68,NaN,68,0,45.0,NaN,68,0.0,...,0.0,0.0,0,0.0,1.0,-1.2,1.0,0,2024-05-09,1
2,0.0,1.0,26,2.0,16,0,13.0,299000.0,36,0.0,...,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-05-30,1
3,0.0,1.0,45,123.0,49,1,158.0,140000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-27,1
4,0.0,1.0,45,123.0,89,0,36.0,170000.0,36,0.0,...,0.0,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-08,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74308,NaN,1.0,45,123.0,30,1,NaN,15000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-15,1
74309,0.0,1.0,45,123.0,89,1,12.0,95000.0,36,0.0,...,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-10,1
74310,0.0,NaN,43,2.0,34,0,5.0,15000.0,36,0.0,...,0.0,0.0,1,0.0,1.0,-1.2,1.0,1,2024-06-27,1
74311,0.0,1.0,45,123.0,90,1,52.0,70000.0,36,0.0,...,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-10,1


In [22]:
pred_0 = model.predict_proba(test_0)[:,1]
pred_0

array([0.01372886, 0.0021477 , 0.18518593, ..., 0.01615292, 0.16930435,
       0.18345018])

In [23]:
pred_1 = model.predict_proba(test_1)[:,1]
pred_1

array([0.02148798, 0.00438424, 0.28135125, ..., 0.02784471, 0.21538041,
       0.21854331])

In [24]:
uplift = pred_1 - pred_0
uplift

array([0.00775911, 0.00223654, 0.09616532, ..., 0.01169179, 0.04607606,
       0.03509312])

In [25]:
sub

,successful_utilization
0,-0.635423
1,-0.884969
2,0.000742
3,0.030837
4,-0.355528
...,...
74308,0.640732
74309,0.933332
74310,-0.085592
74311,-0.886694


In [26]:
sub['successful_utilization'] = uplift
sub

,successful_utilization
0,0.007759
1,0.002237
2,0.096165
3,0.044314
4,0.066033
...,...
74308,0.074372
74309,0.078673
74310,0.011692
74311,0.046076


In [27]:
sub.to_csv('sub.csv')